In [ ]:
import matplotlib.pyplot as plt

from utils import *
import jax.numpy as np

from optimize import *
from regression import LogisticRegression

import math

# Plotting code

This will show a scatterplot. Each array is a list of optimization runs, containing the training and validation loss.

In [ ]:
def scatterplot(saga_losses, sgd_losses):
    plt.scatter(saga_losses[:,0], saga_losses[:,1], label = 'SAGA')
    plt.scatter(sgd_losses[:,0], sgd_losses[:,1], label = 'SGD') 
    
    plt.xlabel('Training loss')
    plt.ylabel('Validation loss')
    plt.title('A comparison of SAGA and SGD in convergence')
    plt.legend()
#

In [ ]:
def gather_data(X_train, Y_train, X_val, Y_val, label_0, label_1):
    train_query = np.logical_or(Y_train==label_0, Y_train==label_1)
    val_query = np.logical_or(Y_val==label_0, Y_val==label_1)

    Y_train_query = Y_train[train_query]
    Y_train_mapped = np.where(Y_train_query == label_0, np.zeros(Y_train_query.shape[0]), np.ones(Y_train_query.shape[0]))

    Y_val_query = Y_val[val_query]
    Y_val_mapped = np.where(Y_val_query == label_0, np.zeros(Y_val_query.shape[0]), np.ones(Y_val_query.shape[0]))

    return X_train[train_query][:2500], Y_train_mapped[:2500], X_val[val_query], Y_val_mapped
#

In [ ]:
dataset = 'cifar10'
X_train_all = np.load('X_'+dataset+'_train.npy')
Y_train_all = np.load('Y_'+dataset+'_train.npy')
X_val_all = np.load('X_'+dataset+'_val.npy')
Y_val_all = np.load('Y_'+dataset+'_val.npy')

cifar10_labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
label_to_ind = dict([(cifar10_labels[i], i) for i in range(len(cifar10_labels))])

In [ ]:
X_train, Y_train, X_val, Y_val = gather_data(X_train_all, Y_train_all, X_val_all, Y_val_all, label_to_ind['horse'], label_to_ind['dog'])

N = X_train.shape[0]
D = X_train.shape[1]

# number of optimization steps
T = 2**14

# fix the seed
seed = 0

n_runs = 50

# run optimization below, with step sizes set to (L = Lipschitz constant):
# * SGD (diminishing): starting step size is 2/L, ending step size is 1e-2/L
# * SAGA (constant): step size is .1/L

# Note: no L2 regularization
lr = LogisticRegression(X_train, Y_train, X_val, Y_val, beta = 0)

# no need to log anything during optimization -> use in place of the loss function
no_logger = lambda x : 0